## 라이브러리 설치

### Mecab

In [ ]:
!pip3 install konlpy

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt-get install curl git

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

### Multiprocess

In [ ]:
!pip3 install multiprocess

### Transformers

In [ ]:
!pip3 install transformers

### PyRouge

In [ ]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path '/content/pyrouge/tools/ROUGE-1.5.5'

In [ ]:
!sudo apt-get install libxml-parser-perl

In [ ]:
%%shell
cd pyrouge/tools/ROUGE-1.5.5/data
rm WordNet-2.0.exc.db # only if exist
cd WordNet-2.0-Exceptions
rm WordNet-2.0.exc.db # only if exist

./buildExeptionDB.pl . exc WordNet-2.0.exc.db
cd ../
ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

### TensorboardX

In [ ]:
!pip3 install tensorboardX

## 라이브러리 Import 

In [11]:
import os
import json
from tqdm import tqdm
import re
import easydict

## 데이터


###원본 데이터 탐색
- 데이터 구성
  - 원문 데이터 40만 건 (신문기사 30만 건, 기고문 6만 건, 잡지기사 1만 건, 법원 판결문 3만 건)을 활용하여 각각 추출요약 40만 건, 생성요약 40만 건, 총 80만 건의 요약문 도출
  - 원문으로부터 변형 없이 그대로 선택된 3개 문장으로 추출요약문 생성
  - 원문의 내용을 바탕으로 재작성된 생성요약문 생성

In [12]:
DATAPATH = '/content/drive/MyDrive/인공지능/추출요약/data/raw_data/train'
filenames = [x for x in os.listdir (DATAPATH) if x.endswith('json')]
filenames.sort()
filenames

['train_original_law.json',
 'train_original_news.json',
 'train_original_opinion.json']

In [13]:
file = filenames[0]
filelocation = os.path.join(DATAPATH, file)

with open(filelocation, 'r') as json_file:
  data = json.load(json_file) ['documents']

In [14]:
data[0]

{'id': '100004',
 'category': '일반행정',
 'size': 'small',
 'char_count': 377,
 'publish_date': '19841226',
 'title': '부당노동행위구제재심판정취소',
 'text': [[{'index': 0,
    'sentence': '원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,',
    'highlight_indices': ''},
   {'index': 1, 'sentence': '노조조합장이 사임한 경우,', 'highlight_indices': ''},
   {'index': 2,
    'sentence': '노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고',
    'highlight_indices': '8,9;68,69'},
   {'index': 3,
    'sentence': '종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고',
    'highlight_indices': ''},
   {'index': 4,
    'sentence': '고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면,',
    'highlight_indices': '0,3'},
   {'index': 5,
    'sentence': '이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.',
    'highlight_indices': '17,21'}]],
 'annotator_id': 3783,
 'document_quality_scores': {'readable': 3,
  'accurate': 3,
 

### 원본 데이터에서 필요한 값 추출
- text와 extractive 추출
  - text : 정규식을 이용해 sentence - highlight_indices 사이의 문장 추출한 후 리스트로 저장
  - extracive : 3줄 요약에 해당하는 문장 index 3개가 저장된 리스트
- 신문기사, 기고문, 법원 판결분 3개로 나누어져 잇는 파일을 하나로 합침
  - train, valid 각각에 대해 수행

#### Train 데이터

In [15]:
DATAPATH = '/content/drive/MyDrive/인공지능/추출요약/data/raw_data/train'
filenames = [x for x in os.listdir (DATAPATH) if x.endswith('json')]
filenames.sort()
filenames

['train_original_law.json',
 'train_original_news.json',
 'train_original_opinion.json']

In [16]:
# list_dic = []

# for file in filenames:
#   filelocation = os.path.join(DATAPATH, file)

#   with open(filelocation, 'r') as json_file:
#     data = json.load(json_file)['documents']

#     for x in tqdm (range(len(data))):
#       text = data[x]['text']
#       text = str(text).replace('"', "'")

#       extractive = data[x]['extractive']
#       for index, value in enumerate(extractive):
#         if value == None:
#           extractive[index] = 0

#       p = re.compile('(?<=sentence\'\: \')(.*?)(?=\'highlight_indices)')
#       texts = p.findall(text)

#       sentences = []
#       for t in texts:
#         sentence = t[:-3]
#         sentences.append(sentence)

#       mydict = {}
#       mydict['text'] = sentences
#       mydict['extractive'] = extractive
#       list_dic.append(mydict)

In [18]:
# len(list_dic)

In [19]:
# with open("/content/drive/MyDrive/인공지능/추출요약/data/raw_data/train.json", 'w') as fh:
#   json.dump(list_dic, fh)

In [20]:
# def list_chunk(lst, n):
#     return [lst[i:i+n] for i in range(0, len(lst), n)]

# data_chunked = list_chunk(data, 32507) ## 전체 데이터를 10개로 분할

# for i, d in enumerate(data_chunked):
#   with open("/content/drive/MyDrive/인공지능/추출요약/data/raw_data/train.{}.json".format(i), 'w') as fh:
#     json.dump(d, fh)

#### val 

In [21]:
DATAPATH = '/content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid'
filenames = [x for x in os.listdir (DATAPATH) if x.endswith('json')]
filenames.sort()
filenames

['valid_original_law.json',
 'valid_original_news.json',
 'valid_original_opinion.json']

In [22]:
# list_dic = []

# for file in filenames:
#   filelocation = os.path.join(DATAPATH, file)

#   with open(filelocation, 'r') as json_file:
#     data = json.load(json_file)['documents']

#     for x in tqdm (range(len(data))):
#       text = data[x]['text']
#       text = str(text).replace('"', "'")

#       extractive = data[x]['extractive']
#       for index, value in enumerate(extractive):
#         if value == None:
#           extractive[index] = 0

#       p = re.compile('(?<=sentence\'\: \')(.*?)(?=\'highlight_indices)')
#       texts = p.findall(text)

#       sentences = []
#       for t in texts:
#         sentence = t[:-3]
#         sentences.append(sentence)

#       mydict = {}
#       mydict['text'] = sentences
#       mydict['extractive'] = extractive
#       list_dic.append(mydict)

In [23]:
# with open("/content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid.json", 'w') as fh:
#   json.dump(list_dic, fh)

## 학습가능한 형태로 데이터 변환
### Tokenizing 후 .json 파일로 저장
- Source 및 Target 설정
  - Source; 원본 데이터의 'text'에 해당
  - Target; 원본 데이터의 'extractive' 리스트의 index에 해당하는 문장을 source에서 추출
- Source 및 Target 을 Tokenizing
  - Tokenizer; MeCab + BertTokenizer
    - 한국어 데이터이므로 MeCab으로 형태소 분석 후 BertTokenizer 사용
  - Tokenizing 후 token 단위로 source 및 target 저장

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()

#### train 데이터

In [25]:
DATAPATH = '/content/drive/MyDrive/인공지능/추출요약/data/raw_data'
filenames = [x for x in os.listdir (DATAPATH) if 'train' in x and x.endswith('json')]
filenames

['train.json']

In [26]:
def data_preprocess(set_name):
  with open("/content/drive/MyDrive/인공지능/추출요약/data/raw_data/{}.json".format(set_name), 'r') as f:
    data = json.load(f)

    list_dic = []
    for x in tqdm(range(len(data))):
      text = data[x]['text']
      extractive = data[x]['extractive']

      sentences = []
      for sentence in text:
        sentence_morph = ' '.join(mecab.morphs(sentence))
        sentences.append(sentence_morph)

      extractives = []
      for e in extractive:
        extractives.append(sentences[e])

      src = [i.split(' ') for i in sentences]
      tgt = [i.split(' ') for i in extractives]

      mydict = {}
      mydict['src'] = src
      mydict['tgt'] = tgt
      list_dic.append(mydict)

    jsonfilelocation = '/content/drive/MyDrive/인공지능/추출요약/data/json_data/' + set_name
    os.makedirs(jsonfilelocation, exist_ok=True)

    temp = []
    DATA_PER_FILE = 50

    for i,a in enumerate(tqdm(list_dic)):
      if (i+1)%DATA_PER_FILE!=0:
        temp.append(a)
      else:
        temp.append(a)
        filename = 'korean.'+ set_name + '.' + str(i//DATA_PER_FILE)+'.json'
        with open(os.path.join(jsonfilelocation, filename), "w", encoding='utf-8') as json_file:
          json.dump(temp, json_file, ensure_ascii=False)
          temp = []

      #마지막에 남은 데이터 있으면 추가로 append
      if len(temp) != 0:
        filename = 'korean.'+ set_name + '.' + str(i//DATA_PER_FILE + 1)+'.json'
        with open(os.path.join(jsonfilelocation, filename), "w", encoding='utf-8') as json_file:
          json.dump(temp, json_file, ensure_ascii=False)

In [27]:
# data_preprocess('train')
# data_preprocess('valid')

### BERT feature 생성 후 .pt 파일로 저장
- format_to_bert 함수 이용
- prepro > data_builder.py
  - BertData() 클래스에서 tokenizer 설정
  - BertTokenizer.from_pretrained('klue/bert-base', strip_accents=False, do_lower_case=False)

In [28]:
import glob
from os.path import join as pjoin
import sys

from multiprocess import Pool
sys.path.append('/content/drive/MyDrive/인공지능/추출요약')
from SRC.prepro.data_builder import _format_to_bert

def format_to_bert(args):
    if (args.dataset != ''):
        datasets = args.dataset
    else:
        datasets = ['train', 'valid', 'test']
    for corpus_type in datasets:
        a_lst = []
        for json_f in glob.glob(pjoin(args.raw_path, corpus_type, '*' + corpus_type + '.*.json')):
            real_name = json_f.split('/')[-1]
            a_lst.append((json_f, args, pjoin(args.save_path, real_name.replace('json', 'bert.pt'))))
        # print(a_lst)
        pool = Pool(args.n_cpus)
        for d in pool.imap(_format_to_bert, a_lst):
            pass

        pool.close()
        pool.join()

In [29]:
# set_name = 'train'

# bertfilelocation = '/content/drive/MyDrive/인공지능/추출요약/data/bert_data/' + set_name
# os.makedirs(bertfilelocation, exist_ok=True)

# args = easydict.EasyDict({
#   "dataset": [set_name], 
#   "raw_path": "/content/drive/MyDrive/인공지능/추출요약/data/json_data/",
#   "save_path": bertfilelocation,
#   "n_cpus":4,
#   "oracle_mode": "greedy",
#   "min_src_ntokens": 5,
#   "max_src_ntokens": 200,
#   "min_nsents": 3,
#   "max_nsents": 100,
# }) 

# format_to_bert(args)

In [30]:
# set_name = 'valid'

# bertfilelocation = '/content/drive/MyDrive/인공지능/추출요약/data/bert_data/' + set_name
# os.makedirs(bertfilelocation, exist_ok=True)

# args = easydict.EasyDict({
#   "dataset": [set_name], 
#   "raw_path": "/content/drive/MyDrive/인공지능/추출요약/data/json_data/",
#   "save_path": bertfilelocation,
#   "n_cpus":4,
#   "oracle_mode": "greedy",
#   "min_src_ntokens": 5,
#   "max_src_ntokens": 200,
#   "min_nsents": 3,
#   "max_nsents": 100,
# })

# format_to_bert(args)

## Model
### 1. 사전 학습 모델 로딩
- models > model_builder.py
  - `Bert()` 클래스에서 model 설정
  - `BertModel.from_pretrained('klue/bert-base')`
- train.py
  - `validate`와 `test` 함수에서 config 설정
  - `BertConfig.from_pretrained('klue/bert-base')`

### 2. 인코더 설정
- models > encoder.py
  - `Classifier()` 클래스
  - `TransformerInterEncoder()` 클래스
  - `RNNEncoder()` 클래스
- models > model_builder.py
  - `Summarizer()` 클래스에서 encoder 설정

## Train
### 1. 학습 loss 함수 설정
- models > trainer.py
  - `__init__` 함수에서 loss 설정
  - `torch.nn.BCELoss(reduction='none')`

### 2. 하이퍼파라미터 설정
- 경로 관련 하이퍼파라미터
  - model_path (Path): Fine-tuning한 모델이 저장되는 경로
  - result_path (Path): 평가 시 추론 결과(candidate)와 정답(gold)이 저장되는 경로
  - temp_dir (Path): 평가 시 ROUGE SCORE 결과가 저장되는 경로
  - log_file (Path): 로그 파일이 저장되는 경로
  - train_from (Path): 학습하고자 하는 체크포인트(pt)가 있을 경우 설정하는 경로
  - bert_data_path (Path): 학습에 사용할 Bert Data(.pt)를 설정하는 경로

- 모델 관련 하이퍼파라미터
  - batch_size (int): A Batch Size (default=1000)
  - use_interval (bool): 학습 시 문서 내에서 여러 문장을 구별하기 위해 Interval Segment Embeddings를 사용하는 지의 여부 (default=True)
  - hidden_size (int): Transformer Hidden Size (default=128)
  - ff_size (int): Feed-forward Filter Size (default=2048)
  - heads (int): Transformer Head 개수 (default=8)
  - inter_layers (int): Transformer Inter Layer 개수 (default=2)
  - rnn_size (int): Encoder RNN RNN Size(default=512)
  - train_steps(int): 학습할 Step 수

- Optimizer 관련 하이퍼파라미터
  - param_init (float): (default=0)
  - param_init_glorot (bool): (default=True)
  - dropout (float): Dropout (default=0.1)
  - optim (str): Optimizer (default='adam')
  - lr (float): Learning Rate (default=2e-3)
  - beta1 (float): Adam Optimizer 관련 Hyper Parameter (default= 0.9)
  - beta2 (float): Adam Optimizer 관련 Hyper Parameter (default=0.999)
  - decay_method (str): Weight Decay (default='noam')
  - warmup_steps (int): Warm-up Steps (default=6000)
  - max_grad_norm (float): Max Gradient Noam (default=0)

- 모델 저장 관련 하이퍼파라미터
  - save_checkpoint steps (int): 모델 저장 주기

- Multi-GPU 관련 하이퍼파라미터
  - world_size (int): GPU 개수
  - visible_gpus (str): GPU 번호
  - gpu_ranks (str): GPU 번호
  - seed (int): (default=666)

- 기타 하이퍼파라미터
  - accum_count (int): (default=2)
  - report_every (int): (default=50)
  - recall_eval (bool): (default=false)
  - report_rouge (bool): (default=false)
  - block_trigram (bool): (default=true)











### 3. Multi-GPU 설정
- train.py
  - `multi_main` 과 `run` 함수 참조
  - multiprocessing 중 spawn 사용

```
if(args.world_size>1):
  multi_main(args)
```
- distributed.py
  - `multi_init` 함수 참조
  - 분산학습 사용
- Colab 은 Multi-GPU를 지원하지 않으므로, 개인 장비에서 가능하다면 실행 추천


### 4. ROUGE 설정
- models > trainer.py
  - `Trainer()` 클래스의 `test` 함수 참조
- others > utils.py
  - `test_rouge`, `rouge_results_to_str` 함수 사용

### 5. 최종 학습 코드
- train.py에서 mode를 train으로 선택

In [31]:
!nvidia-smi

Thu Mar 23 12:09:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    49W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Classifier - 논문에는 5만 step

In [32]:
# logdirlocation = '/content/drive/MyDrive/인공지능/추출요약/LOG/KLUE'
# os.makedirs(logdirlocation, exist_ok=True)

# !python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
#     -mode train \
#     -encoder classifier \
#     -dropout 0.1 \
#     -bert_data_path /content/drive/MyDrive/인공지능/추출요약/data/bert_data/train/korean \
#     -model_path /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_classifier \
#     -lr 2e-3 \
#     -visible_gpus 0 \
#     -gpu_ranks 0 \
#     -world_size 1 \
#     -report_every 1000\
#     -save_checkpoint_steps 100 \
#     -batch_size 1000 \
#     -decay_method noam \
#     -train_steps 1000 \
#     -accum_count 2 \
#     -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_classifier.txt \
#     -use_interval true \
#     -warmup_steps 200


#### 5-2 RNN

In [33]:
# logdirlocation = '/content/drive/MyDrive/인공지능/추출요약/LOG/KLUE'
# os.makedirs(logdirlocation, exist_ok=True)

# !python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
#   -mode train \
#   -encoder rnn \
#   -dropout 0.1 \
#   -bert_data_path /content/drive/MyDrive/인공지능/추출요약/data/bert_data/train/korean \
#   -model_path /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_rnn \
#   -lr 2e-3 \
#   -visible_gpus 0 \
#   -gpu_ranks 0 \
#   -world_size 1 \
#   -report_every 1000\
#   -save_checkpoint_steps 100 \
#   -batch_size 1000 \
#   -decay_method noam \
#   -train_steps 1000 \
#   -accum_count 2 \
#   -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_rnn.txt \
#   -use_interval true \
#   -warmup_steps 200 \
#   -rnn_size 768 # 신경써서 해야함

#### 5-3 Transformer

In [34]:
# logdirlocation = '/content/drive/MyDrive/인공지능/추출요약/LOG/KLUE'
# os.makedirs(logdirlocation, exist_ok=True)

# !python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
#   -mode train \
#   -encoder transformer \
#   -dropout 0.1 \
#   -bert_data_path /content/drive/MyDrive/인공지능/추출요약/data/bert_data/train/korean \
#   -model_path /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer \
#   -lr 2e-3 \
#   -visible_gpus 0 \
#   -gpu_ranks 0 \
#   -world_size 1 \
#   -report_every 1000\
#   -save_checkpoint_steps 100 \
#   -batch_size 1000 \
#   -decay_method noam \
#   -train_steps 1000 \
#   -accum_count 2 \
#   -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_transformer.txt \
#   -use_interval true \
#   -warmup_steps 200 \
#   -ff_size 2048 \
#   -inter_layers 2 \
#   -heads 8

### 최종모델 선별 및 50,000 epoch 학습

In [36]:
logdirlocation = '/content/drive/MyDrive/인공지능/추출요약/LOG/KLUE'
os.makedirs(logdirlocation, exist_ok=True)

!python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
  -mode train \
  -encoder transformer \
  -dropout 0.1 \
  -bert_data_path /content/drive/MyDrive/인공지능/추출요약/data/bert_data/train/korean \
  -model_path /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer_result \
  -lr 2e-3 \
  -visible_gpus 0 \
  -gpu_ranks 0 \
  -world_size 1 \
  -report_every 1000\
  -save_checkpoint_steps 10000 \
  -batch_size 1000 \
  -decay_method noam \
  -train_steps 50000 \
  -accum_count 2 \
  -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_transformer_result.txt \
  -use_interval true \
  -warmup_steps 10000 \
  -ff_size 2048 \
  -inter_layers 2 \
  -heads 8

<RootLogger root (INFO)>
[2023-03-23 12:11:40,044 INFO] Device ID 0
[2023-03-23 12:11:40,045 INFO] Device cuda
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[2023-03-23 12:

## 6. 최종 평가 코드
- train.py에서 mode를 test로 선택
- valid 데이터 사용
  - 이때 데이터 이름을 test로 변경해야 진행되므로 `korean.valid.{}.bert.pt` 데이터를 `korean.test.{}.bert.pt` 데이터로 이름 변경
- **주의**
  - test 시 pyrouge 라이브러리를 통해 rouge score를 계산하게 되는데, Colab 상에서는 dependency 문제로 인해 에러 발생
  - 위의 Install libraries 가이드대로 pyrouge 설치 후 SRC, 데이터, 모델을 로컬 디스크에 다운로드 후 로컬에서 실행해야 함


### 6-1. Classifier

In [ ]:
logdirlocation = '/content/drive/MyDrive/인공지능/추출요약/LOG/KLUE'
os.makedirs(logdirlocation, exist_ok=True)

resdirlocation = '/content/drive/MyDrive/인공지능/추출요약/RESULT'
os.makedirs(resdirlocation, exist_ok=True)

tmpdirlocation = '/content/drive/MyDrive/인공지능/추출요약/TEMP'
os.makedirs(tmpdirlocation, exist_ok=True)


!python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
  -mode test \
  -bert_data_path /content/drive/MyDrive/인공지능/추출요약/data/bert_data/valid2/korean \
  -model_path /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_classifier \
  -result_path /content/drive/MyDrive/인공지능/추출요약/RESULT/ \
  -temp_dir /content/drive/MyDrive/인공지능/추출요약/TEMP/ \
  -visible_gpus 0 \
  -gpu_ranks 0 \
  -world_size 1 \
  -batch_size 30000 \
  -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_classifier.txt \
  -test_from /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_classifier/model_step_1000.pt

### 6-2. RNN

In [ ]:
logdirlocation = '/content/drive/MyDrive/인공지능/추출요약/LOG/KLUE'
os.makedirs(logdirlocation, exist_ok=True)

resdirlocation = '/content/drive/MyDrive/인공지능/추출요약/RESULT'
os.makedirs(resdirlocation, exist_ok=True)

tmpdirlocation = '/content/drive/MyDrive/인공지능/추출요약/TEMP'
os.makedirs(tmpdirlocation, exist_ok=True)


!python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
  -mode test \
  -bert_data_path /content/drive/MyDrive/인공지능/추출요약/data/bert_data/valid2/korean \
  -model_path /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_rnn \
  -result_path /content/drive/MyDrive/인공지능/추출요약/RESULT/ \
  -temp_dir /content/drive/MyDrive/인공지능/추출요약/TEMP/ \
  -visible_gpus 0 \
  -gpu_ranks 0 \
  -world_size 1 \
  -batch_size 30000 \
  -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_rnn.txt \
  -test_from /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_rnn/model_step_1000.pt

### 6-3. Transformer

In [ ]:
logdirlocation = '/content/drive/MyDrive/인공지능/추출요약/LOG/KLUE'
os.makedirs(logdirlocation, exist_ok=True)

resdirlocation = '/content/drive/MyDrive/인공지능/추출요약/RESULT'
os.makedirs(resdirlocation, exist_ok=True)

tmpdirlocation = '/content/drive/MyDrive/인공지능/추출요약/TEMP'
os.makedirs(tmpdirlocation, exist_ok=True)


!python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
  -mode test \
  -bert_data_path /content/drive/MyDrive/인공지능/추출요약/data/bert_data/valid_2/korean \
  -model_path /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer \
  -result_path /content/drive/MyDrive/인공지능/추출요약/RESULT/ \
  -temp_dir /content/drive/MyDrive/인공지능/추출요약/TEMP/ \
  -visible_gpus 0 \
  -gpu_ranks 0 \
  -world_size 1 \
  -batch_size 30000 \
  -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_transformer.txt \
  -test_from /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer/model_step_1000.pt

## 7. 세줄 요약 추론 코드 구현
- train.py에서 mode를 inference로 선택
- models > trainer.py의 `summ` 함수 참조


## 8. 추론 결과에 대한 정량적/정성적 평가 
- 가장 성능이 좋았던 transformer encoder로 추론
- valid.json 중 5개 검증

In [ ]:
!unzip /content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid/sample.zip -d /content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid

Archive:  /content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid/sample.zip
  inflating: /content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid/valid_0.txt  
  inflating: /content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid/valid_1.txt  
  inflating: /content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid/valid_2.txt  


In [ ]:
!python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
  -mode inference \
  -visible_gpus -1 \
  -gpu_ranks -1 \
  -world_size 0 \
  -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_transformer_result.txt \
  -test_from /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer_result/model_step_50000.pt \
  -input_text /content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid/valid_0.txt

2023-02-23 11:19:56.239661: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 11:19:56.402448: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-23 11:19:57.166169: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-23 11:19:57.166278: W tensorflow/compiler/xla/stream_executor

In [ ]:
!python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
  -mode inference \
  -visible_gpus -1 \
  -gpu_ranks -1 \
  -world_size 0 \
  -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_transformer_result.txt \
  -test_from /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer_result/model_step_50000.pt \
  -input_text /content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid/valid_1.txt

2023-02-23 11:20:18.260713: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 11:20:18.421688: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-23 11:20:19.174458: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-23 11:20:19.174552: W tensorflow/compiler/xla/stream_executor

In [ ]:
!python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
  -mode inference \
  -visible_gpus -1 \
  -gpu_ranks -1 \
  -world_size 0 \
  -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_transformer_result.txt \
  -test_from /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer_result/model_step_50000.pt \
  -input_text /content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid/valid_2.txt

2023-02-23 11:20:27.565763: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 11:20:27.723791: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-23 11:20:28.668332: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-23 11:20:28.668435: W tensorflow/compiler/xla/stream_executor

## 성능개선방항
|개선사항|이유|진행률(%)|
|:-----:|:-----:|:-----:|
|Data Augmentation|법률문서 낮은 score||
|5만 step로 학습|테스트로 1000만 학습||
|Transformer로 서비스 구현|Transformer가 가장 성능이 좋음||
|RoBERTa, ELECTRA등 고려|BERT보다 좋은 성능 모델 존재||



## 평가지표 테스트

In [ ]:
with open('/content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid/valid_2.txt', 'r', encoding='utf-8') as f:
    text_list = f.read().splitlines()

In [ ]:
reference = ['[ 박재원 기자 ] \'대한민국 5G 홍보대사\'를 자처한 문재인 대통령은 \\"넓고, 체증 없는 \'통신 고속도로\'가 5G\\"라며 \\"대한민국의 대전환이 이제 막 시작됐다\\"고 기대감을 높였다.', "하지만 당 일각에선 \"총선기획단장을 비롯한 당직 인선부터 쇄신 의지를 보여야 한다\"는 비판의 목소리가 나온다.","대신 4 차 산업혁명의 거센 파고를 헤쳐나갈 전문성을 갖춘 젊고 유능한 인재들을 널리 구해야 하다 "]
hypothesis  = ['문 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 \\"5G 시대는 우리가 생각하고, 만들면 그것이 세계 표준이 되는 시대\\"라며 \\"5G는 대한민국 혁신성장의 인프라\\"라고 강조했다.', "산업화 시대에 고속도로가 우리 경제의 '대동맥' 역할을 했듯, 5G가 4차 산업혁명 시대의 고속도로가 돼 새로운 기회를 열어 줄 것이란 설명이다.", '[ 박재원 기자 ] \'대한민국 5G 홍보대사\'를 자처한 문재인 대통령은 \\"넓고, 체증 없는 \'통신 고속도로\'가 5G\\"라며 \\"대한민국의 대전환이 이제 막 시작됐다\\"고 기대감을 높였다.']

In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
reference = ' '.join(reference)
hypothesis = ' '.join(hypothesis)

In [ ]:
print(reference)
print(hypothesis)

[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 \"넓고, 체증 없는 '통신 고속도로'가 5G\"라며 \"대한민국의 대전환이 이제 막 시작됐다\"고 기대감을 높였다. 하지만 당 일각에선 "총선기획단장을 비롯한 당직 인선부터 쇄신 의지를 보여야 한다"는 비판의 목소리가 나온다. 대신 4 차 산업혁명의 거센 파고를 헤쳐나갈 전문성을 갖춘 젊고 유능한 인재들을 널리 구해야 하다 
문 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 \"5G 시대는 우리가 생각하고, 만들면 그것이 세계 표준이 되는 시대\"라며 \"5G는 대한민국 혁신성장의 인프라\"라고 강조했다. 산업화 시대에 고속도로가 우리 경제의 '대동맥' 역할을 했듯, 5G가 4차 산업혁명 시대의 고속도로가 돼 새로운 기회를 열어 줄 것이란 설명이다. [ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 \"넓고, 체증 없는 '통신 고속도로'가 5G\"라며 \"대한민국의 대전환이 이제 막 시작됐다\"고 기대감을 높였다.


In [ ]:
scores = rouge.get_scores(hypothesis, reference, avg=True)
scores['rouge-1']['f']

0.3931623882241216

- 일단 f와 r까지 추출하는 코드를 구현했다.
- 이제는 모든 데이터를 리스트에 담아서 하는 방법을 고안해야한다.
- 그런데 또 하나의 문제가 생긴다 어떻게 모든 데이터들을 txt에 담아서 예측하고 평가하지?

In [ ]:
DATAPATH = '/content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid'
filenames = [x for x in os.listdir (DATAPATH) if x.endswith('json')]
filenames.sort()
filenames

['valid_original_law.json',
 'valid_original_news.json',
 'valid_original_opinion.json']

In [ ]:
list_dic = []

for file in filenames:
  filelocation = os.path.join(DATAPATH, file)

  with open(filelocation, 'r') as json_file:
    data = json.load(json_file)['documents']

    for x in tqdm (range(len(data))):
      text = data[x]['text']
      text = str(text).replace('"', "'")

      extractive = data[x]['extractive']
      for index, value in enumerate(extractive):
        if value == None:
          extractive[index] = 0

      p = re.compile('(?<=sentence\'\: \')(.*?)(?=\'highlight_indices)')
      texts = p.findall(text)

      sentences = []
      for t in texts:
        sentence = t[:-3]
        sentences.append(sentence)

      mydict = {}
      mydict['text'] = sentences
      mydict['extractive'] = extractive
      list_dic.append(mydict)

100%|██████████| 7008/7008 [00:03<00:00, 2211.11it/s]


In [ ]:
len(list_dic)

40134

In [ ]:
len(list_dic)

40134

In [ ]:
list_dic[0]

{'text': ['[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,',
  '처분 등이 있은 날부터 1년을 경과하면',
  '제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다.',
  '[2] 일반적으로 행정처분에 효력기간이 정하여져 있는 경우에는 그 기간의 경과로 그 행정처분의 효력은 상실되며,',
  '다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.',
  '[3] 사도개설허가에서 정해진 공사기간 내에 사도로 준공검사를 받지 못한 경우,',
  '이 공사기간을 사도개설허가 자체의 존속기간(유효기간)으로 볼 수 없다는 이유로 사도개설허가가 당연히 실효되는 것은 아니라고 한 사례.'],
 'extractive': [0, 4, 6]}

In [ ]:
selected_texts = []
for j in tqdm(range(len(list_dic))):
    selected_texts.append([list_dic[j]['text'][i] for i in list_dic[j]['extractive']])


100%|██████████| 40134/40134 [00:00<00:00, 416714.19it/s]


In [ ]:
len(selected_texts)

40134

- 이제 문제는 text데이터를 우에 전부다 모델에 넣고 학습을 시키냐이다. 

In [ ]:
input_text2 = []
for i in tqdm(range(len(list_dic))):
    input_text2.append(list_dic[i]['text'])

100%|██████████| 40134/40134 [00:00<00:00, 1269373.79it/s]


In [ ]:
len(input_text2)

40134

In [ ]:
input_text2[0]

['[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,',
 '처분 등이 있은 날부터 1년을 경과하면',
 '제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다.',
 '[2] 일반적으로 행정처분에 효력기간이 정하여져 있는 경우에는 그 기간의 경과로 그 행정처분의 효력은 상실되며,',
 '다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.',
 '[3] 사도개설허가에서 정해진 공사기간 내에 사도로 준공검사를 받지 못한 경우,',
 '이 공사기간을 사도개설허가 자체의 존속기간(유효기간)으로 볼 수 없다는 이유로 사도개설허가가 당연히 실효되는 것은 아니라고 한 사례.']

In [ ]:
!python /content/drive/MyDrive/인공지능/추출요약/SRC/train.py \
  -mode inference \
  -visible_gpus -1 \
  -gpu_ranks -1 \
  -world_size 0 \
  -log_file /content/drive/MyDrive/인공지능/추출요약/LOG/KLUE/bert_transformer_result.txt \
  -test_from /content/drive/MyDrive/인공지능/추출요약/MODEL/KLUE/bert_transformer_result/model_step_50000.pt \
  -input_text /content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid/valid_2.txt

2023-02-24 06:06:44.876326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 06:06:46.057748: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-24 06:06:46.057893: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-24 06:06:46.057920: W tensorfl

- 평가는 함수를 불러와서 여기에서 직접해야겠다
- 그리고 불러올 때는 변경된 코드를 불러와야 된다.
- 어떻게 변경하는지는 다시 코드를 면밀히 살펴야 한다.
- 아래 부분은 로컬에서 하는게 낫겠다 어차피 gpu도 필요없는 작업이고 여기서 하면 가만있어서 컴퓨팅 나가고 또 런타임 초기화해야지 코드가 먹힐때가 있다.

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/인공지능/추출요약/SRC")
from train import inference2

In [ ]:
test = inference2

In [ ]:
test(input_text2[0])

여기오나?
['[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,', '처분 등이 있은 날부터 1년을 경과하면', '제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다.', '[2] 일반적으로 행정처분에 효력기간이 정하여져 있는 경우에는 그 기간의 경과로 그 행정처분의 효력은 상실되며,', '다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.', '[3] 사도개설허가에서 정해진 공사기간 내에 사도로 준공검사를 받지 못한 경우,', '이 공사기간을 사도개설허가 자체의 존속기간(유효기간)으로 볼 수 없다는 이유로 사도개설허가가 당연히 실효되는 것은 아니라고 한 사례.']
여기오나?
new_test2 :  ['[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,', '처분 등이 있은 날부터 1년을 경과하면', '제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다.', '[2] 일반적으로 행정처분에 효력기간이 정하여져 있는 경우에는 그 기간의 경과로 그 행정처분의 효력은 상실되며,', '다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.', '[3] 사도개설허가에서 정해진 공사기간 내에 사도로 준공검사를 받지 못한 경우,', '

NameError: ignored

In [ ]:
input_text2[0]

['[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,',
 '처분 등이 있은 날부터 1년을 경과하면',
 '제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다.',
 '[2] 일반적으로 행정처분에 효력기간이 정하여져 있는 경우에는 그 기간의 경과로 그 행정처분의 효력은 상실되며,',
 '다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.',
 '[3] 사도개설허가에서 정해진 공사기간 내에 사도로 준공검사를 받지 못한 경우,',
 '이 공사기간을 사도개설허가 자체의 존속기간(유효기간)으로 볼 수 없다는 이유로 사도개설허가가 당연히 실효되는 것은 아니라고 한 사례.']